# Data collection

### Load packages/set globals

In [1]:
import tweepy
consumer_key = "51MI8RrYmzO4btCKG4Qb5uqAa"
consumer_secret = "ajpPv3Ag0NvMEQLBIwiPyDyU78BbLZn8IS1gTba4x9ZOHNPMNM"
access_token = "3004471069-VDbNpT9NO0QOtiqKZXkoH5Flv4MArCflIYImXjn"
access_token_secret = "sP6KMjPZXxYAnaae8bOiauLjCVnx8bzWkBk4KU1iZBxdl"

In [2]:
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 

n = 3 # Tweet count

### Basic API functionality

In [3]:
# Print tweets from my home timeline
for status in tweepy.Cursor(api.home_timeline).items(n):
    # Process a single status
    print(status.text)
#     print(status._json)

"A lot of excitement for me." @DuaneBrown76 shared how coming to Seattle was the perfect scenario. 

📰 |… https://t.co/kFng4RmnFE
When it comes to critical-thinking skills, Washington students need help, new analysis suggests (By @pgcornwell) https://t.co/rD5whNm77C
This just in: The Colts announced Andrew Luck will be placed on Injured Reserve for the rest of the 2017 season. https://t.co/NiQUfF665i


In [4]:
# Getting tweets from the New York Times user
for tweet in api.user_timeline(id="nytimes", count=n):
    # printing the text stored inside the tweet object
    print(tweet.text)

Janet Yellen will be the first person in nearly 40 years to serve no more than one 4-year term as head of the Fed https://t.co/DDnx16d1iZ
Now that her children are grown, Michelle Pfeiffer has returned in a head-snapping way this year… https://t.co/rncgVri57q
What did Bernie Sanders learn about Canada's health system? Doctors like free health care as much as patients do. https://t.co/9MRsphQjcu


In [5]:
# get tweets by keyword
for tweet in api.search(q="stock", lang="en")[:n]:
    # printing the text stored inside the tweet object
    print(tweet.user.screen_name,"Tweeted:",tweet.text)

byr_ne Tweeted: RT @TelegramJames: Sometimes it's fun to see who else used the same stock photo. @NLEmployer's "Another Way Forward" and ... https://t.co/K…
cleepohl Tweeted: RT @RepAdamSchiff: What kind of president calls our justice system a joke and laughing stock, with “so-called” judges? The worst President…
jo_kasprzak Tweeted: RT @MiaFarrow: Reminder: Attorney General Jeff Sessions is the head of our so-called 'laughing stock' justice system


# Tweet processing

## Tokenization

### Default Tokenizer

In [6]:
from nltk.tokenize import word_tokenize

In [7]:
# note a default tokenizer doesn't do a good job
tweets = api.search(q="stock", lang="en")
tweet  = tweets[0].text
print(word_tokenize(tweet))

['RT', '@', 'TelegramJames', ':', 'Sometimes', 'it', "'s", 'fun', 'to', 'see', 'who', 'else', 'used', 'the', 'same', 'stock', 'photo', '.', '@', 'NLEmployer', "'s", '``', 'Another', 'Way', 'Forward', "''", 'and', '...', 'https', ':', '//t.co/K…']


### Tweet specific tokenizer

In [14]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def chunk(s):
    return tokens_re.findall(s)
 
def tokenize(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [16]:
print(tokenize(tweet.text))
# ['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']

['RT', '@MiaFarrow', ':', 'Reminder', ':', 'Attorney', 'General', 'Jeff', 'Sessions', 'is', 'the', 'head', 'of', 'our', 'so-called', "'", 'laughing', 'stock', "'", 'justice', 'system']


## Stop terms

In [10]:
from nltk.corpus import stopwords
import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']

In [20]:
def filt(terms, stop):
    return [term for term in terms if term not in stop]

In [21]:
import operator 
import json
from collections import Counter
 
count_all = Counter()
for tweet in tweets[:3]:
    # Create a list with all the terms
    terms_all = filt(tokenize(tweet.text), stop)
    # Update the counter
    count_all.update(terms_all)
    # Print the first 5 most frequent words
    print(tweet.text)
    print(count_all.most_common(5))

RT @TelegramJames: Sometimes it's fun to see who else used the same stock photo. @NLEmployer's "Another Way Forward" and ... https://t.co/K…
[('photo', 1), ('…', 1), ('else', 1), ('Another', 1), ('Sometimes', 1)]
RT @RepAdamSchiff: What kind of president calls our justice system a joke and laughing stock, with “so-called” judges? The worst President…
[('…', 2), ('RT', 2), ('stock', 2), ('photo', 1), ('else', 1)]
RT @MiaFarrow: Reminder: Attorney General Jeff Sessions is the head of our so-called 'laughing stock' justice system
[('RT', 3), ('stock', 3), ('…', 2), ('justice', 2), ('system', 2)]


In [23]:
import operator 
import json
from collections import Counter
 
count_all = Counter()
for tweet in tweets[:3]:
    # Create a list with all the terms
    terms_all = tokenize(tweet.text)
    # Update the counter
    count_all.update(terms_all)
    # Print the first 5 most frequent words
    print(tweet.text)
    print(count_all.most_common(5))

RT @TelegramJames: Sometimes it's fun to see who else used the same stock photo. @NLEmployer's "Another Way Forward" and ... https://t.co/K…
[('.', 4), ('"', 2), ('photo', 1), ('…', 1), ('else', 1)]
RT @RepAdamSchiff: What kind of president calls our justice system a joke and laughing stock, with “so-called” judges? The worst President…
[('.', 4), ('…', 2), ('RT', 2), ('"', 2), ('stock', 2)]
RT @MiaFarrow: Reminder: Attorney General Jeff Sessions is the head of our so-called 'laughing stock' justice system
[('.', 4), (':', 4), ('RT', 3), ("'", 3), ('stock', 3)]


In [ ]:
# # Count terms only once, equivalent to Document Frequency
# terms_single = set(terms_all)
# # Count hashtags only
# terms_hash = [term for term in preprocess(tweet['text']) 
#               if term.startswith('#')]
# # Count terms only (no hashtags, no mentions)
# terms_only = [term for term in preprocess(tweet['text']) 
#               if term not in stop and
#               not term.startswith(('#', '@'))] 
#               # mind the ((double brackets))
#               # startswith() takes a tuple (not a list) if 
#               # we pass a list of inputs

In [24]:
from nltk import bigrams

In [25]:
terms_bigram = bigrams(stop)

In [27]:
list(terms_bigram)

[('i', 'me'),
 ('me', 'my'),
 ('my', 'myself'),
 ('myself', 'we'),
 ('we', 'our'),
 ('our', 'ours'),
 ('ours', 'ourselves'),
 ('ourselves', 'you'),
 ('you', 'your'),
 ('your', 'yours'),
 ('yours', 'yourself'),
 ('yourself', 'yourselves'),
 ('yourselves', 'he'),
 ('he', 'him'),
 ('him', 'his'),
 ('his', 'himself'),
 ('himself', 'she'),
 ('she', 'her'),
 ('her', 'hers'),
 ('hers', 'herself'),
 ('herself', 'it'),
 ('it', 'its'),
 ('its', 'itself'),
 ('itself', 'they'),
 ('they', 'them'),
 ('them', 'their'),
 ('their', 'theirs'),
 ('theirs', 'themselves'),
 ('themselves', 'what'),
 ('what', 'which'),
 ('which', 'who'),
 ('who', 'whom'),
 ('whom', 'this'),
 ('this', 'that'),
 ('that', 'these'),
 ('these', 'those'),
 ('those', 'am'),
 ('am', 'is'),
 ('is', 'are'),
 ('are', 'was'),
 ('was', 'were'),
 ('were', 'be'),
 ('be', 'been'),
 ('been', 'being'),
 ('being', 'have'),
 ('have', 'has'),
 ('has', 'had'),
 ('had', 'having'),
 ('having', 'do'),
 ('do', 'does'),
 ('does', 'did'),
 ('did', 'doi